In [1]:
import tensorflow.keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
%matplotlib inline

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Image Data Generator w/ no augmentation
#Scaling for pixels
piece_train_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_test_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_valid_datagen = ImageDataGenerator(
    rescale = 1./255)


#Flow data from directory

piece_train_iter = piece_train_datagen.flow_from_directory(
    directory = '../data/color_data/train',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

piece_test_iter = piece_test_datagen.flow_from_directory(
    directory = '../data/color_data/test',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle=False,
    seed=42
)

piece_valid_iter = piece_valid_datagen.flow_from_directory(
    directory = '../data/color_data/valid',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

Found 1227 images belonging to 3 classes.
Found 335 images belonging to 3 classes.
Found 331 images belonging to 3 classes.


In [3]:
#Define NN architecture

from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential

piece_model = Sequential()
piece_model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(135, 135, 1)))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(GlobalAveragePooling2D())
piece_model.add(Dense(3, activation='softmax'))


piece_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 135, 135, 16)      80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 67, 67, 16)        0         
_________________________________________________________________
batch_normalization_v2 (Batc (None, 67, 67, 16)        64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 67, 67, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
batch_normalization_v2_1 (Ba (None, 33, 33, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 33, 64)        8

In [4]:
# compile the model
piece_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

STEP_SIZE_TRAIN = piece_train_iter.n/piece_train_iter.batch_size
STEP_SIZE_VALID = piece_valid_iter.n/piece_valid_iter.batch_size



In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='baseline_model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
piece_hist = piece_model.fit_generator(generator=piece_train_iter, 
                          steps_per_epoch=STEP_SIZE_TRAIN, 
                          validation_data=piece_valid_iter, 
                          validation_steps=STEP_SIZE_VALID,
                          epochs=100, 
                          callbacks=[checkpointer], 
                          verbose=2)

Epoch 1/100

Epoch 00001: val_loss improved from inf to 1.18820, saving model to baseline_model.weights.best.hdf5
39/38 - 8s - loss: 0.5580 - accuracy: 0.7710 - val_loss: 1.1882 - val_accuracy: 0.4411
Epoch 2/100

Epoch 00002: val_loss did not improve from 1.18820
39/38 - 5s - loss: 0.3236 - accuracy: 0.8941 - val_loss: 1.2599 - val_accuracy: 0.1511
Epoch 3/100

Epoch 00003: val_loss did not improve from 1.18820
39/38 - 5s - loss: 0.2748 - accuracy: 0.8989 - val_loss: 1.4006 - val_accuracy: 0.1511
Epoch 4/100

Epoch 00004: val_loss did not improve from 1.18820
39/38 - 4s - loss: 0.2477 - accuracy: 0.9177 - val_loss: 1.4874 - val_accuracy: 0.1511
Epoch 5/100

Epoch 00005: val_loss did not improve from 1.18820
39/38 - 4s - loss: 0.2415 - accuracy: 0.9234 - val_loss: 1.6234 - val_accuracy: 0.1511
Epoch 6/100

Epoch 00006: val_loss did not improve from 1.18820
39/38 - 5s - loss: 0.1859 - accuracy: 0.9381 - val_loss: 2.0986 - val_accuracy: 0.1511
Epoch 7/100

Epoch 00007: val_loss did not i

Epoch 52/100

Epoch 00052: val_loss did not improve from 0.02820
39/38 - 7s - loss: 0.0136 - accuracy: 0.9959 - val_loss: 0.7939 - val_accuracy: 0.8792
Epoch 53/100

Epoch 00053: val_loss did not improve from 0.02820
39/38 - 7s - loss: 0.0024 - accuracy: 0.9992 - val_loss: 0.2361 - val_accuracy: 0.9245
Epoch 54/100

Epoch 00054: val_loss improved from 0.02820 to 0.02181, saving model to baseline_model.weights.best.hdf5
39/38 - 8s - loss: 0.0036 - accuracy: 0.9992 - val_loss: 0.0218 - val_accuracy: 0.9970
Epoch 55/100

Epoch 00055: val_loss did not improve from 0.02181
39/38 - 7s - loss: 0.0244 - accuracy: 0.9943 - val_loss: 0.0793 - val_accuracy: 0.9849
Epoch 56/100

Epoch 00056: val_loss did not improve from 0.02181
39/38 - 7s - loss: 0.0083 - accuracy: 0.9967 - val_loss: 0.0520 - val_accuracy: 0.9909
Epoch 57/100

Epoch 00057: val_loss did not improve from 0.02181
39/38 - 7s - loss: 0.0032 - accuracy: 0.9992 - val_loss: 0.0543 - val_accuracy: 0.9909
Epoch 58/100

Epoch 00058: val_los

In [8]:
#Reset test iterator
STEP_SIZE_TEST = piece_test_iter.n/piece_test_iter.batch_size
piece_test_iter.reset()
# load the weights that yielded the best validation accuracy
piece_model.load_weights('baseline_model.weights.best.hdf5')
# evaluate and print test accuracy
score = piece_model.evaluate_generator(generator=piece_test_iter,steps=STEP_SIZE_TEST)
print('\n', 'Test accuracy:', score[1])

ValueError: Shapes (64, 3) and (64, 7) are incompatible

In [ ]:
piece_test_iter.reset()
piece_pred = piece_model.predict_generator(piece_test_iter,steps=STEP_SIZE_TEST,verbose=1)

In [ ]:
predicted_class_indices=np.argmax(piece_pred,axis=1)
print(predicted_class_indices)

In [ ]:
labels = (piece_test_iter.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
truth =  [labels[k] for k in piece_test_iter.classes]

In [ ]:
[labels[k] for k in piece_test_iter.classes]


In [ ]:
piece_test_iter.filenames

In [ ]:
filenames=piece_test_iter.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Truth": truth,
                      "Predictions":predictions})

In [ ]:
results

In [ ]:
from sklearn.metrics import classification_report
class_report = classification_report(piece_test_iter.classes,predicted_class_indices)
print(class_report)

In [ ]:
from sklearn.metrics import confusion_matrix
cmatrix = confusion_matrix(piece_test_iter.classes,predicted_class_indices)
print(cmatrix)
print(labels)

In [ ]:
from sklearn.metrics import log_loss
log_loss_result = log_loss(piece_test_iter.classes,piece_pred)
print(log_loss_result)

In [ ]:
plt.plot(piece_hist.history['acc'])
plt.plot(piece_hist.history['val_acc'])
plt.title('Baseline Color Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(piece_hist.history['loss'])
plt.plot(piece_hist.history['val_loss'])
plt.title('Baseline Color Model Log Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
import seaborn as sns
ax = sns.heatmap(cmatrix, annot=True, xticklabels=['black','empty','white'],yticklabels=['black','empty','white'])